## Import libraries

In [43]:
from shutil import which
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException

from selenium import webdriver
import time
import pandas as pd
import os
import time
import requests
import numpy as np
import pandas as pd
import math

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

### Glassdoor API

There is no API for jobs and reviews, but scraping is possible with Python libraries like Selenium or BeautifulSoup.

In [30]:
# Scraper based on https://github.com/kelvinxuande/glassdoor-scraper.git
scraper1_output = pd.read_csv(r"C:\Users\Phillip\dschenaina\glassdoor-scraper\src\output\output_11-09-2022.csv",encoding ="ISO-8859-1", delimiter = ",")
scraper1_output.head(10)

,companyName,company_starRating,company_offeredRole,company_roleLocation,listing_jobDesc,requested_url
0,Mastercard,4.3,Software Engineer,Singapore,Are you passionate about software engineering?...,https://www.glassdoor.com/partner/jobListing.h...
1,Dyson,3.1,Software Engineer,Singapore,\nDevelop and maintain real time embedded soft...,https://www.glassdoor.com/partner/jobListing.h...
2,Shopee,3.8,Frontend Software Engineer - Marketplace,Singapore,\nTake ownership of various business features ...,https://www.glassdoor.com/partner/jobListing.h...
3,Superstar Teacher Pte Ltd,NaN,Assistant Software Engineer / Software Engineer,Singapore,"\nWrite, modify, and debug web applications us...",https://www.glassdoor.com/partner/jobListing.h...
4,NTUC ENTERPRISE NEXUS CO-OPERATIVE LIMITED,3.3,Software Engineer,Singapore,4 years of professional experience. Strong hol...,https://www.glassdoor.com/partner/jobListing.h...
5,SEARCH INDEX PTE. LTD.,5.0,Software Engineer (Entry Level Welcome / MNC),Singapore,Responsible for Software Development Life Cycl...,https://www.glassdoor.com/partner/jobListing.h...
6,HASHSTACS PTE. LTD.,4.1,Frontend Software Engineer (Remote),Singapore,B.S. or M.S. in Computer Science or equivalent...,https://www.glassdoor.com/partner/jobListing.h...
7,NCS,3.8,Software Engineer,Singapore,Partner with stakeholders to understand busine...,https://www.glassdoor.com/partner/jobListing.h...
8,Rolls-Royce,4.0,Software Engineer (Python/SQL),Singapore,\nWorking with colleagues to understand and im...,https://www.glassdoor.com/partner/jobListing.h...
9,Anotech Singapore,3.9,Software Test Engineer (Junior),Singapore,Monday to Friday,https://www.glassdoor.com/partner/jobListing.h...


In [53]:
# Scraper based on https://github.com/rohan-benjamin/Glassdoor-Scraper-Final.git
def fetch_jobs(keyword, num_pages):
    options = Options()
    options.add_argument("window-size=1920,1080")
    #Enter your chromedriver.exe path below
    chrome_path = r".\chromedriver.exe"
    driver = webdriver.Chrome(executable_path=chrome_path, options=options)
    driver.get("https://www.glassdoor.co.in/Job/Home/recentActivity.htm")
    search_input = driver.find_element("id","sc.keyword")
    search_input.send_keys(keyword)
    search_input.send_keys(Keys.ENTER)
    time.sleep(2)
    
    company_name = []
    job_title = []
    salary_est = []
    location = []
    job_description = []
    salary_estimate = []
    company_size = []
    company_type = []
    company_sector = []
    company_industry = []
    company_founded = []
    company_revenue = []
    
    #Set current page to 1
    current_page = 1     
    time.sleep(3)
    
    while current_page <= num_pages:   
        
        done = False
        while not done:
            job_cards = driver.find_elements("xpath","//article[@id='MainCol']//ul/li[@data-adv-type='GENERAL']")
            for card in job_cards:
                try: 
                    card.click()
                    time.sleep(1)
                except:
                    continue

                #Closes the signup prompt
                try:
                    driver.find_element("xpath",".//span[@class='SVGInline modal_closeIcon']").click()
                    time.sleep(2)
                except (NoSuchElementException, ElementClickInterceptedException):
                    time.sleep(2)
                    pass

                #Expands the Description section by clicking on Show More
                try:
                    driver.find_element("xpath","//div[@class='css-t3xrds e856ufb2']").click()
                    time.sleep(1)
                except NoSuchElementException:
                    card.click()
                    print(str(current_page) + '#ERROR: no such element')
                    time.sleep(30)
                    driver.find_element("xpath","//div[@class='css-t3xrds e856ufb2']").click()
                except ElementNotInteractableException:
                    card.click()
                    driver.implicitly_wait(30)
                    print(str(current_page) + '#ERROR: not interactable')
                    driver.find_element("xpath","//div[@class='css-t3xrds e856ufb2']").click()

                #Scrape 

                try:
                    company_name.append(driver.find_element("xpath","//div[@class='css-xuk5ye e1tk4kwz5']").text)
                except:
                    company_name.append("#N/A")
                    pass

                try:
                    job_title.append(driver.find_element("xpath","//div[@class='css-1j389vi e1tk4kwz2']").text)
                except:
                    job_title.append("#N/A")
                    pass

                try:
                    location.append(driver.find_element("xpath","//div[@class='css-56kyx5 e1tk4kwz1']").text)
                except:
                    location.append("#N/A")
                    pass

                try:
                    job_description.append(driver.find_element("xpath","//div[@id='JobDescriptionContainer']").text)
                except:
                    job_description.append("#N/A")
                    pass

                try:
                    salary_estimate.append(driver.find_element("xpath","//div[@class='css-y2jiyn e2u4hf18']").text)
                except:
                    salary_estimate.append("#N/A")
                    pass
                
                try:
                    company_size.append(driver.find_element("xpath","//div[@id='CompanyContainer']//span[text()='Size']//following-sibling::*").text)
                except:
                    company_size.append("#N/A")
                    pass
                
                try:
                    company_type.append(driver.find_element("xpath","//div[@id='CompanyContainer']//span[text()='Type']//following-sibling::*").text)
                except:
                    company_type.append("#N/A")
                    pass
                    
                try:
                    company_sector.append(driver.find_element("xpath","//div[@id='CompanyContainer']//span[text()='Sector']//following-sibling::*").text)
                except:
                    company_sector.append("#N/A")
                    pass
                    
                try:
                    company_industry.append(driver.find_element("xpath","//div[@id='CompanyContainer']//span[text()='Industry']//following-sibling::*").text)
                except:
                    company_industry.append("#N/A")
                    pass
                    
                try:
                    company_founded.append(driver.find_element("xpath","//div[@id='CompanyContainer']//span[text()='Founded']//following-sibling::*").text)
                except:
                    company_founded.append("#N/A")
                    pass
                    
                try:
                    company_revenue.append(driver.find_element("xpath","//div[@id='CompanyContainer']//span[text()='Revenue']//following-sibling::*").text)
                except:
                    company_revenue.append("#N/A")
                    pass
                    
                done = True
                
       # Moves to the next page         
        if done:
            print(str(current_page) + ' ' + 'out of' +' '+ str(num_pages) + ' ' + 'pages done')
            driver.find_element("xpath","//span[@alt='next-icon']").click()   
            current_page = current_page + 1
            time.sleep(4)

    driver.close()
    df = pd.DataFrame({'company': company_name, 
    'job title': job_title,
    'location': location,
    'job description': job_description,
    'salary estimate': salary_estimate,
    'company_size': company_size,
    'company_type': company_type,
    'company_sector': company_sector,
    'company_industry' : company_industry, 'company_founded' : company_founded, 'company_revenue': company_revenue})
    df.head(10)
    df.to_csv(keyword + '.csv')            
                     
fetch_jobs("Data Engineer", 1)

C:\Users\Phillip\AppData\Local\Temp\ipykernel_18076\20735192.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path, options=options)


ElementClickInterceptedException: Message: element click intercepted: Element <div class="css-t3xrds e856ufb2">...</div> is not clickable at point (1203, 262). Other element would receive the click: <div class="css-dufhjo e1tk4kwz0">...</div>
  (Session info: chrome=105.0.5195.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x0109DF13+2219795]
	Ordinal0 [0x01032841+1779777]
	Ordinal0 [0x00F4423D+803389]
	Ordinal0 [0x00F799D4+1022420]
	Ordinal0 [0x00F778C4+1013956]
	Ordinal0 [0x00F754AB+1004715]
	Ordinal0 [0x00F74117+999703]
	Ordinal0 [0x00F69B76+957302]
	Ordinal0 [0x00F8E7FC+1107964]
	Ordinal0 [0x00F694B4+955572]
	Ordinal0 [0x00F8EA14+1108500]
	Ordinal0 [0x00F9F192+1175954]
	Ordinal0 [0x00F8E616+1107478]
	Ordinal0 [0x00F67F89+950153]
	Ordinal0 [0x00F68F56+954198]
	GetHandleVerifier [0x01392CB2+3040210]
	GetHandleVerifier [0x01382BB4+2974420]
	GetHandleVerifier [0x01136A0A+565546]
	GetHandleVerifier [0x01135680+560544]
	Ordinal0 [0x01039A5C+1808988]
	Ordinal0 [0x0103E3A8+1827752]
	Ordinal0 [0x01

By using NLP methods, we could extract opinions about a companies Work/Life balance, culture and values, salaries, ...
Example: https://towardsdatascience.com/what-your-employees-are-saying-online-about-you-text-mining-using-python-72fe2a9bd615

![alt text](sc2.png "Title")

## Stepstone API

Stepstone also does not have a public programming API, but can be extracted with mentioned Python libraries. Possible approach: http://datalab.bui.haw-hamburg.de/projects/project_01/BA_Wardatzky_Kathrin_161107.pdf

## Kununu API

Kununu does have an API, but intended to be used by companies (not for free). There is one tool to specifically scrape kununu company reviews: https://github.com/TheWoops/Web-Scraping.git
The test was successful, but it requires some training to understand how to use it. Kununu can also be scaped with mentioned Python libraries.

## Bundesbank API

The "Deutsche Bundesbank" offers current statistical data of the Bundesbank in the form of time series and real-time data: https://www.bundesbank.de/en/statistics/time-series-databases
This could provide relevant macroeconomic information, e.g.:
![alt text](sc.png "Title")


## LinkedIn API

One can download LinkedIn connections and other data to analyze interests of employees: https://towardsdatascience.com/mining-data-on-linkedin-9b70681b1467

## Jobfeed API

Jobfeed offers a oublic API for developers to gather data (https://www.jobfeed.de/api/v3/help). They collect real-time job market data specifically for HR purposes. Accounts are limited to 4 requests/second and 30 requests/minute.

## Google datasets

Number of datasets for different areas of research: https://datasetsearch.research.google.com/

## Scrapingrobot API

Up to 5000 scraping requests per month: https://docs.scrapingrobot.com/reference/basic-usage?_ga=2.74448466.1796286644.1662903980-1566943732.1662903980

## Indeed API

Scrapable using Python libraries: https://www.pycodemates.com/2022/01/Indeed-jobs-scraping-with-python-bs4-selenium-and-pandas.html

In [52]:
def get_current_url(url, job_title, location):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(3)
    driver.find_element('xpath','//*[@id="text-input-what"]').send_keys(job_title)
    time.sleep(3)
    driver.find_element('xpath','//*[@id="text-input-where"]').send_keys(location)
    time.sleep(3)
    driver.find_element('xpath','/html/body/div').click()
    time.sleep(3)
    try:
        driver.find_element('xpath','//*[@id="jobsearch"]/button').click()
    except:
        driver.find_element('xpath','//*[@id="whatWhereFormId"]/div[3]/button').click()
    current_url = driver.current_url

    return current_url 

current_url = get_current_url('https://in.indeed.com/','Data Scientist',"Bengaluru")
print(current_url)

def scrape_job_details(url):
    
    
    resp = requests.get(url)
    content = BeautifulSoup(resp.content, 'lxml')
   
    jobs_list = []    
    for post in content.select('.job_seen_beacon'):
        print(post)
        try:
            data = {
                "job_title":post.select('.jobTitle')[0].get_text().strip(),
                "company":post.select('.companyName')[0].get_text().strip(),
                "rating":post.select('.ratingNumber')[0].get_text().strip(),
                "location":post.select('.companyLocation')[0].get_text().strip(),
                "date":post.select('.date')[0].get_text().strip(),
                "job_desc":post.select('.job-snippet')[0].get_text().strip()
                
            }
        except IndexError:
            continue          
        jobs_list.append(data)
    dataframe = pd.DataFrame(jobs_list)
    
    return dataframe

scrape_job_details(current_url)

https://in.indeed.com/jobs?q=Data%20Scientist&l=Bengaluru&from=searchOnHP&vjk=1f2f17aad584b2c7


""


## Gehalt.de

https://www.gehalt.de/gehaltsvergleich